# data

> Functionality for efficiently accessing data in WEAVE fits files

In [ ]:
# |default_exp data

In [ ]:
# | hide
from nbdev.showdoc import show_doc

In [ ]:
# |export

import os
import sys
import time
from functools import partial, wraps
from glob import glob
from multiprocessing import Pool
from typing import Callable

import dask
import numpy as np
import xarray as xr
from astropy.io import fits
from astropy.table import Table
from tqdm import tqdm

The `data` module provides functions for locating and reading WEAVE data as `xarray` `Dataset`s.

## Accessing FITS tables as xarray Datasets via cached netCDF files

The approach of `qagmire` is to access WEAVE data stored on disk in FITS files on disk. To analyse this large, multi-dimensional dataset we utilize [`xarray`](https://docs.xarray.dev), making use of its ability to use `dask` to perform computations in parallel in a memory efficient and scaleable manner. To make this work we need to write the data to disk in a suitable format: netCDF files, then load those files as an `xarray.Dataset`. This functionality is implemented in `FITStoDataset`, which is used to wrap simpler functions which focus on reading data from a single FITS file into a convenient `Dataset`. In general, the aim is for the resulting `Dataset`s to preserve the structure in the FITS files. However, there are some cases where it is sensible to rearrange the data into a more convenient and/or efficient format.

In [ ]:
# |export


def _read_single(
    read_function: Callable[[str], xr.Dataset],  # function to read a FITS file
    fn: str,  # filename of FITS file to read
):
    """Read a FITS file to an xarray Dataset using the given read function."""
    ds = read_function(fn)
    if ds:
        fn_base = os.path.splitext(os.path.basename(fn))[0]
        ds = ds.expand_dims({"filename": [fn_base]})
        try:
            run = fits.getval(fn, "RUN")
            ds = ds.assign_coords(RUN=("filename", [run]))
        except KeyError:
            pass
        try:
            camera = fits.getval(fn, "CAMERA")
            camera = camera.replace("WEAVE", "")
            ds = ds.assign_coords(CAMERA=("filename", [camera]))
        except KeyError:
            pass
        try:
            mjd = np.round(fits.getval(fn, "MJD-OBS"), 4)
            ds = ds.assign_coords(MJD=("filename", [mjd]))
        except KeyError:
            pass
        try:
            obid = fits.getval(fn, "OBID")
            ds = ds.assign_coords(OBID=("filename", [obid]))
        except KeyError:
            pass
    else:
        table = read_function.__name__.replace("read_", "")
        print(f"Warning: cannot read {table} for file {fn}.")
    return ds


def _single_via_netcdf(
    read_function: Callable[[str], xr.Dataset],  # function to read a FITS file
    fn: str,  # filename of FITS file to read
    netcdf_store,
    update_cache,
):
    """Transform a FITS file to netCDF using the given read function.

    Returns the path of a netCDF file stored in `nedcdf_store`, containing the `Dataset` resulting
    from calling `read_function` with the supplied FITS filename `fn`. If the netCDF file already
    exists, the filename is immediately returned.
    """
    fn_netcdf = os.path.join(netcdf_store, *os.path.normpath(fn).split(os.sep)[-3:])
    table = read_function.__name__.replace("read_", "")
    fn_netcdf = os.path.splitext(fn_netcdf)[0]
    fn_netcdf = f"{fn_netcdf}_{table}.nc"
    if not os.path.exists(fn_netcdf) or update_cache:
        ds = _read_single(read_function, fn)
        if ds:
            os.makedirs(os.path.dirname(fn_netcdf), exist_ok=True)
            ds.to_netcdf(fn_netcdf, format="NETCDF4", engine="netcdf4")
            ds.close()
        else:
            fn_netcdf = None
    return fn_netcdf


class FITStoDataset:
    """Access multiple FITS tables as an xarray Dataset, optionally via cached netCDF files.

    For each FITS table or image we wish to read, we will write a `read_*(fn)` function
    which reads the table from the single provided filename `fn` and returns a `Dataset`.
    Wrapping such a function with an instance of this class adapts the function to take
    a list of FITS filenames and return a `Dataset`. If `cache=True`, the Dataset is
    lazily loaded data from a cache of netCDF files. The cache is stored in the
    `netcdf_store` folder defined when the instance is initialised.

    If `cache=True`, when the wrapped function is initially run, it repeatedly calls
    `single_via_netcdf` to apply the original `read_*` function to each FITS filename and
    save each resulting `Dataset` as a netCDF file, then opens them together and returns a
    combined, distributed `Dataset`. If a previously converted file is found in the
    `netcdf_store`, then the original `read_*` function is skipped and the netCDF file loaded
    directly. This caching can vastly increase the speed of subsequent calls.
    If `n_processes > 1`, which it is by default, then this reading and caching is performed
    in parallel using `n_processes` processes.

    Although instances of this class can be used as a decorator, doing so with
    `n_processes > 1` will lead to an exception due to pickling issues with multiprocessing.
    Instead they should be used to wrap functions, without replacing the original function name.
    For example,
    ```
    to_dataset = FITStoDataset()

    def _class_spec_reader(fn):
        ...

    read_class_spec = to_dataset(_class_spec_reader)
    ```

    If a source FITS file is changed, the corresponding files in `netcdf_store` can simply
    be deleted and they will be recreated on the next call of the decorated `read_*`
    function.

    If `cache=False`, then the data is always read from the specified FITS files, combined
    and returned as an in-memory Dataset. This may be faster when dealing with lots of small
    files.

    If `update_cache=False`, then existing cache files are not read, but are recreated.

    """

    def __init__(
        self,
        cache=True,  # cache the dataset to netCDF files
        netcdf_store: str | None = None,  # folder in which to store the netCDF files
        progress=True,  # display a progress bar
        update_cache=False,  # read FITS files and recreate netCDF files, no effect if cache=False
        n_processes=8,  # how many subprocesses to use
    ):
        """Create a decorator that can extend a `read_*` function to multiple files.

        If no `netcdf_store` is provided it first checks for a `NETCDF_STORE` environment
        variable and falls back to a folder called `netcdf_store` in the user's home folder.
        """
        self.cache = cache
        self.update_cache = update_cache
        self.n_processes = n_processes

        if netcdf_store is not None:
            self.netcdf_store = netcdf_store
        else:
            default = "/beegfs/weavelofar/netcdf_store"
            self.netcdf_store = os.environ.get("NETCDF_STORE", default)

        if progress:
            if cache:
                desc = "Locating and converting where necessary"
            else:
                desc = "Reading files"
            self.progress = partial(tqdm, desc=desc, file=sys.stdout)
        else:
            self.progress = lambda x: x

    def __call__(
        self,
        read_function: Callable[[str], xr.Dataset],  # function to read a FITS file
    ):
        """Extend the functionality of `read_function` to multiple files.

        The wrapped `read_function` is adapted to take a list of FITS filenames and
        return a `Dataset`, which lazily loads data from a cache of netCDF files if
        `self.cache=True` (the default).
        """

        @wraps(read_function)
        def wrapper(fns):
            if self.cache:
                read = partial(
                    _single_via_netcdf,
                    read_function,
                    netcdf_store=self.netcdf_store,
                    update_cache=self.update_cache,
                )
            else:
                read = partial(_read_single, read_function)

            if self.n_processes > 1:
                results = []
                try:
                    with Pool(self.n_processes) as p:
                        with self.progress(total=len(fns)) as pbar:
                            for f in p.imap_unordered(read, fns):
                                results.append(f)
                                pbar.update()
                except PermissionError:
                    raise PermissionError(
                        "Cannot access the NetCDF file. Ensure any previously "
                        "created Datasets are closed, e.g. ds.close()"
                    )
            else:
                results = [read(fn) for fn in self.progress(fns)]

            results = [f for f in results if f is not None]

            if self.cache:
                print("Reading netCDF files... ", end="")
                start = time.perf_counter()
                data = xr.open_mfdataset(
                    results,
                    parallel=True,
                    combine="nested",
                    coords="minimal",
                    concat_dim="filename",
                    engine="netcdf4",
                )
            else:
                print("Creating Dataset... ", end="")
                start = time.perf_counter()
                data = xr.concat(results, dim="filename", coords="minimal")
            dt = time.perf_counter() - start
            print(f"took {dt:.2f} s. Size is {data.nbytes * 2**-20:.3f} Mb")
            return data

        return wrapper

### Configure dask parallelism

To avoid errors, we need to use dask single-threaded.

In [ ]:
# |exports
# |output false

dask.config.set(scheduler="single-threaded")

<dask.config.set>

To speed up running calculations on large datasets, we can run a set of workers on a single node. There are also ways to easily [leverage multiple nodes](https://docs.dask.org/en/stable/deploying.html). However, most of the time is spent reading in the NetCDF data, which does not seem to make use of the worker nodes. Note that reading the original FITS files, and caching them to NetCDF, is (by default) parallelised using `multiprocessing`.

In [ ]:
# from dask.distributed import Client
# client = Client(n_workers=8, threads_per_worker=1)

### Create the `to_dataset` and `to_dataset_without_cache` wrapping functions

These are used to wrap all the `read_*` functions defined below. The function `to_dataset` is an instance of `FITStoDataset` with all the default behaviour described above.

For datasets consisting of small amounts of data per FITS file, it appears to be more efficient to simply read from the FITS files every time. In such cases, we use `to_dataset_without_cache`, an instance of `FITStoDataset` with caching disabled.

Note that both of must be used to wrap functions, with the result being assigned a different name to the original. They cannot be used as decorators, unless `n_processes=1`.

In [ ]:
# |exports

to_dataset = FITStoDataset()
to_dataset_without_cache = FITStoDataset(cache=False)

Existing cache files will be used when using the `qagmire.data` module. However, if running this notebook, the following line means we always update the cache, such that the timings reflect the original reading and conversion process.

In [ ]:
to_dataset = FITStoDataset(update_cache=True)

## Locating WEAVE FITS files

Here we define some functions to get lists of WEAVE FITS filenames.

In [ ]:
# |export


def _is_lowres(fn):
    """Check the header of FITS file `fn` to determine if it is low-resolution."""
    try:
        lowres = "LR" in fits.getval(fn, "RES-OBS")
    except KeyError:
        lowres = "LOWRES" in fits.getval(fn, "MODE")
    return lowres


def get_weave_files(
    level="*",  # pattern to match to the file level, e.g. raw, L1, L2
    filetype="*",  # pattern to match to the file type, e.g. single, stack
    date="*",  # pattern to match to the date in format yyyymmdd
    runid="*",  # pattern to match to the runid
    lowres=True,  # select low-res files, or high-res if False
):
    """Get a list of matching WEAVE files."""
    if level != "raw":
        filetype += "_"
    pattern = f"{level}/{date}/{filetype}*{runid}*.fit*"
    pattern = os.path.join(os.environ["WEAVEIO_ROOTDIR"], pattern)
    files = glob(pattern)
    files.sort()
    if lowres:
        files = [fn for fn in files if _is_lowres(fn)]
    else:
        files = [fn for fn in files if not _is_lowres(fn)]
    return files


def get_lr_raw_files(
    date="*",  # pattern to match to the date in format yyyymmdd
    runid="*",  # pattern to match to the runid
):
    return get_weave_files(level="raw", date=date, runid=runid, lowres=True)


def get_lr_l1_single_files(
    date="*",  # pattern to match to the date in format yyyymmdd
    runid="*",  # pattern to match to the runid
):
    return get_weave_files(
        level="L1", filetype="single", date=date, runid=runid, lowres=True
    )


def get_lr_l2_stack_files(
    date="*",  # pattern to match to the date in format yyyymmdd
    runid="*",  # pattern to match to the runid
):
    return get_weave_files(
        level="L2", filetype="stack", date=date, runid=runid, lowres=True
    )

In [ ]:
# |export


def _read_fits_columns(
    fn: str,  # the filename of the FITS file to read
    ext: str,  # the name of the extension containing the table to read
    limit_precision=False,  # convert all float64 columns to float32
    index: str | None = None,  # remove rows where this column is masked
):
    """Read a FITS table to a dict of arrays and convert endianness."""
    cols = dict(Table.read(fn, ext, unit_parse_strict="silent"))
    cols = {c: cols[c].newbyteorder().byteswap() for c in cols}
    if limit_precision:
        for c in list(cols):
            if cols[c].dtype.type is np.float64:
                if np.can_cast(np.max(np.abs(cols[c])), np.float32):
                    cols[c] = cols[c].astype(np.float32)
    if index is not None:
        ok = ~cols[index].mask
        cols = {c: cols[c][ok] for c in cols}
    return cols

## Raw files

These contain the raw observations.

We will read some simulated files to show examples.

In [ ]:
lr_raw_files = get_lr_raw_files(date="2017*")
print(len(lr_raw_files), "low-res raw files")

120 low-res raw files


In [ ]:
raw_hdus = fits.open(lr_raw_files[0])

WEAVE raw files contain six extensions:

In [ ]:
print([hdu.name for hdu in raw_hdus])

['PRIMARY', 'RED1_DATA', 'RED2_DATA', 'FIBTABLE', 'GUIDINFO', 'METINFO']


### PRIMARY

The PRIMARY extension contains only a header with lots of information about the observation.

In [ ]:
raw_hdus["PRIMARY"].header[:15]

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
COMMENT -------- Start of the CAMERA Packet -------                             
RUN     =              1003313                                                  
IRAFNAME= 'r1003313'           / redir r2840373.fit > r1003313                  
DETECTOR= 'WVRED   '           / Selected by inference                          
CCDSPEED= 'SLOW    '                                                            
CCDXBIN =                    1                                                  
CCDYBIN =                    1                                                  
CCDSUM  = '1 1     '                                                            
CCDTEMP =    132.05553788667

In [ ]:
# |export


def _primary_header_reader(fn):
    """Read the primary header as a Dataset, stripping comments."""
    hdr = fits.getheader(fn, "PRIMARY")
    for key in hdr:
        if key == "" or "COMM" in key:
            del hdr[key]
    return xr.Dataset(hdr)


read_primary_header = to_dataset_without_cache(_primary_header_reader)

In [ ]:
# |hide
show_doc(read_primary_header, name="read_primary_header", title_level=4)

---

[source](https://github.com/bamford/qagmire/blob/main/qagmire/data.py#L309){target="_blank" style="float:right; font-size:smaller"}

#### read_primary_header

>      read_primary_header (fn)

Read the primary header as a Dataset, stripping comments.

In [ ]:
raw_primary_header = read_primary_header(lr_raw_files)

Reading files: 100%|██████████| 120/120 [00:01<00:00, 103.26it/s]
Creating Dataset... took 1.69 s. Size is 0.600 Mb


In [ ]:
raw_primary_header

<xarray.Dataset>
Dimensions:   (filename: 120)
Coordinates:
  * filename  (filename) object 'r1003316' 'r1003319' ... 'r1004152' 'r1004151'
    RUN       (filename) int64 1003316 1003319 1003315 ... 1004152 1004151
    CAMERA    (filename) <U4 'BLUE' 'RED' 'RED' 'RED' ... 'RED' 'BLUE' 'RED'
    OBID      (filename) int64 3802 3802 3802 3802 3802 ... 3936 3936 3936 3936
    MJD       (filename) float64 5.781e+04 5.781e+04 ... 5.803e+04 5.803e+04
Data variables: (12/410)
    SIMPLE    (filename) bool True True True True True ... True True True True
    BITPIX    (filename) int64 8 8 8 8 8 8 8 8 8 8 8 8 ... 8 8 8 8 8 8 8 8 8 8 8
    NAXIS     (filename) int64 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    EXTEND    (filename) bool True True True True True ... True True True True
    IRAFNAME  (filename) <U8 'r1003316' 'r1003319' ... 'r1004152' 'r1004151'
    DETECTOR  (filename) <U6 'WVBLUE' 'WVRED' 'WVRED' ... 'WVBLUE' 'WVRED'
    ...        ...
    CCNAME17  (filename) <U1 '' '' '' '' '' '' '' '' ... '' '' '' '' '' '' '' ''
    CCNAME18  (filename) <U1 '' '' '' '' '' '' '' '' ... '' '' '' '' '' '' '' ''
    CCNAME19  (filename) <U1 '' '' '' '' '' '' '' '' ... '' '' '' '' '' '' '' ''
    CCNAME20  (filename) <U1 '' '' '' '' '' '' '' '' ... '' '' '' '' '' '' '' ''
    CHECKSUM  (filename) <U16 '8CFaA9EV2AEZ89EZ' ... 'HXMnIULkHULkHULk'
    DATASUM   (filename) <U10 '         0' '         0' ... '         0'

### DATA

The RED1_DATA, RED2_DATA and BLUE1_DATA, BLUE2_DATA extensions contain raw imaging of the spectra.

In [ ]:
# |export


def _raw_data_reader(fn):
    """Read the *_DATA from a WEAVE RAW FITS file as a Dataset."""
    hdus = fits.open(fn)
    for h in hdus:
        if h.name.endswith("1_DATA"):
            counts1 = xr.DataArray(h.data)
        elif h.name.endswith("2_DATA"):
            counts2 = xr.DataArray(h.data)
    return xr.Dataset({"counts1": counts1, "counts2": counts2})


read_raw_data = to_dataset(_raw_data_reader)

In [ ]:
# |hide
show_doc(read_raw_data, name="read_raw_data", title_level=4)

---

[source](https://github.com/bamford/qagmire/blob/main/qagmire/data.py#L321){target="_blank" style="float:right; font-size:smaller"}

#### read_raw_data

>      read_raw_data (fn)

Read the *_DATA from a WEAVE RAW FITS file as a Dataset.

In [ ]:
raw_data = read_raw_data(lr_raw_files)

Locating and converting where necessary: 100%|██████████| 120/120 [00:29<00:00,  4.02it/s]
Reading netCDF files... took 2.59 s. Size is 17888.973 Mb


In [ ]:
raw_data

<xarray.Dataset>
Dimensions:   (filename: 120, dim_0: 6160, dim_1: 6344)
Coordinates:
  * filename  (filename) <U8 'r1003316' 'r1003317' ... 'r1004151' 'r1004152'
    RUN       (filename) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    CAMERA    (filename) <U4 dask.array<chunksize=(1,), meta=np.ndarray>
    MJD       (filename) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    OBID      (filename) int64 dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: dim_0, dim_1
Data variables:
    counts1   (filename, dim_0, dim_1) uint16 dask.array<chunksize=(1, 6160, 6344), meta=np.ndarray>
    counts2   (filename, dim_0, dim_1) uint16 dask.array<chunksize=(1, 6160, 6344), meta=np.ndarray>

It is a good idea to close a `Dataset` when you are finished with it, as otherwise other processes may not be able to access the same underlying files.

In [ ]:
raw_data.close()

### FIBTABLE

The FIBTABLE extension contains information about the fibre allocations.

Code is included to display example content of FITS tables, but commented out as it does not display well online.

In [ ]:
# Table.read(raw_hdus["FIBTABLE"])

In [ ]:
# |export


def _fibre_table_reader(fn):
    """Read the FIBTABLE from a WEAVE RAW FITS file as a Dataset.

    All quantities are indexed by the `APS_ID` of the fibre.

    All column names are mde uppercase for consistency.
    """
    cols = _read_fits_columns(fn, "FIBTABLE", index="FIBREID")
    cols = {c.upper(): cols[c] for c in cols}
    if not cols:
        return None
    coords = dict(APS_ID=cols.pop("FIBREID"))
    for c in cols:
        dims = ["APS_ID"]
        cols[c] = xr.Variable(dims, cols[c], attrs={"unit": str(cols[c].unit)})
    return xr.Dataset(cols, coords)


read_fibre_table = to_dataset_without_cache(_fibre_table_reader)

In [ ]:
# |hide
show_doc(read_fibre_table, name="read_fibre_table", title_level=4)

---

[source](https://github.com/bamford/qagmire/blob/main/qagmire/data.py#L335){target="_blank" style="float:right; font-size:smaller"}

#### read_fibre_table

>      read_fibre_table (fn)

Read the FIBTABLE from a WEAVE RAW FITS file as a Dataset.

All quantities are indexed by the `APS_ID` of the fibre.

All column names are mde uppercase for consistency.

In [ ]:
raw_fibre_table = read_fibre_table(lr_raw_files)

Reading files: 100%|██████████| 120/120 [00:01<00:00, 82.81it/s]
Creating Dataset... took 0.94 s. Size is 22.617 Mb


In [ ]:
raw_fibre_table

<xarray.Dataset>
Dimensions:    (APS_ID: 1008, filename: 120)
Coordinates:
  * APS_ID     (APS_ID) int16 0 1 2 3 4 5 6 ... 1002 1003 1004 1005 1006 1007
  * filename   (filename) object 'r1003314' 'r1003319' ... 'r1004151' 'r1004152'
    RUN        (filename) int64 1003314 1003319 1003320 ... 1004151 1004152
    CAMERA     (filename) <U4 'BLUE' 'RED' 'BLUE' 'RED' ... 'RED' 'RED' 'BLUE'
    MJD        (filename) float64 5.781e+04 5.781e+04 ... 5.803e+04 5.803e+04
    OBID       (filename) int64 3802 3802 3802 3802 3802 ... 3936 3936 3936 3936
Data variables: (12/36)
    CNAME      (filename, APS_ID) object nan nan nan nan nan ... nan nan nan nan
    FIBRERA    (filename, APS_ID) float64 nan nan nan nan ... nan nan nan nan
    FIBREDEC   (filename, APS_ID) float64 nan nan nan nan ... nan nan nan nan
    XPOSITION  (filename, APS_ID) float32 nan nan nan nan ... nan nan nan nan
    YPOSITION  (filename, APS_ID) float32 nan nan nan nan ... nan nan nan nan
    STATUS     (filename, APS_ID) object nan nan nan nan nan ... nan nan nan nan
    ...         ...
    MAG_GG     (filename, APS_ID) float32 nan nan nan nan ... nan nan nan nan
    EMAG_GG    (filename, APS_ID) float32 nan nan nan nan ... nan nan nan nan
    MAG_BP     (filename, APS_ID) float32 nan nan nan nan ... nan nan nan nan
    EMAG_BP    (filename, APS_ID) float32 nan nan nan nan ... nan nan nan nan
    MAG_RP     (filename, APS_ID) float32 nan nan nan nan ... nan nan nan nan
    EMAG_RP    (filename, APS_ID) float32 nan nan nan nan ... nan nan nan nan

### GUIDINFO

The GUIDINFO extension contains info about the guiding. Currently not sure of the best way to organise this data.

In [ ]:
# Table.read(raw_hdus["GUIDINFO"])

### METINFO

The METINFO extension contains meteographical information. Currently not sure of the best way to organise this data.

In [ ]:
# Table.read(raw_hdus["METINFO"], unit_parse_strict="silent")

## L1 files

These contain lower-level processed data products. There are `single` files, which contain info for a single exposure, and `stack` files, which contain the same info for stacked exposures.

We will read some simulated single files to show examples.

In [ ]:
lr_l1_single_files = get_lr_l1_single_files(date="2017*")
print(len(lr_l1_single_files), "low-res L1 single files")

60 low-res L1 single files


In [ ]:
l1_hdus = fits.open(lr_l1_single_files[0])

WEAVE L1 single files contain seven extensions:

In [ ]:
print([hdu.name for hdu in l1_hdus])

['PRIMARY', 'RED_DATA', 'RED_IVAR', 'RED_DATA_NOSS', 'RED_IVAR_NOSS', 'RED_SENSFUNC', 'FIBTABLE']


### PRIMARY

The PRIMARY extension contains only a header with lots of information about the observation.

In [ ]:
l1_hdus["PRIMARY"].header[:15]

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                    8 / number of bits per data pixel                  
NAXIS   =                    0 / number of data axes                            
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
COMMENT -------- Start of the CAMERA Packet -------                             
RUN     =              1003317                                                  
IRAFNAME= 'r1003317'           / redir r2840376.fit > r1003317                  
DETECTOR= 'WVRED   '           / Selected by inference                          
CCDSPEED= 'SLOW    '                                                            
CCDXBIN =                    1                                                  
CCDYBIN =                   

In [ ]:
l1_primary_header = read_primary_header(lr_l1_single_files)

Reading files: 100%|██████████| 60/60 [00:00<00:00, 67.54it/s]
Creating Dataset... took 1.23 s. Size is 0.377 Mb


In [ ]:
l1_primary_header

<xarray.Dataset>
Dimensions:   (filename: 60)
Coordinates:
  * filename  (filename) object 'single_1003317' ... 'single_1004149'
    RUN       (filename) int64 1003317 1003330 1003329 ... 1004150 1004149
    CAMERA    (filename) <U4 'RED' 'BLUE' 'RED' 'BLUE' ... 'BLUE' 'BLUE' 'RED'
    OBID      (filename) int64 3802 3900 3900 3802 3802 ... 4407 3936 3936 3936
    MJD       (filename) float64 5.781e+04 5.781e+04 ... 5.803e+04 5.803e+04
Data variables: (12/447)
    SIMPLE    (filename) bool True True True True True ... True True True True
    BITPIX    (filename) int64 8 8 8 8 8 8 8 8 8 8 8 8 ... 8 8 8 8 8 8 8 8 8 8 8
    NAXIS     (filename) int64 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    EXTEND    (filename) bool True True True True True ... True True True True
    IRAFNAME  (filename) <U8 'r1003317' 'r1003330' ... 'r1004150' 'r1004149'
    DETECTOR  (filename) <U6 'WVRED' 'WVBLUE' 'WVRED' ... 'WVBLUE' 'WVRED'
    ...        ...
    NCOMB     (filename) int64 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1
    PROV0000  (filename) <U18 'single_1003317.fit' ... 'single_1004149.fit'
    CHECKSUM  (filename) <U16 'FdA6IZ83FdA3FZ53' ... 'cbhAeaZ6cafAcaZ3'
    DATASUM   (filename) <U1 '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    DELZPGRA  (filename) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    CALDATE   (filename) int64 20161225 20161225 20161225 ... 20161225 20161225

### DATA and SENSFUNC

The RED/BLUE_DATA, RED/BLUE_IVAR, RED/BLUE_DATA_NOSS, RED/BLUE_IVAR_NOSS and RED_BLUE_SENSFUNC extensions contain the reduced binned spectra and their inverse variance, with and without sky subtraction, plus the sensitivity function. Implementation TBD, but I think it makes sense for all of these to be stored in a single Dataset.

In [ ]:
# |export


def _l1_data_reader(fn):
    """Read the data from a WEAVE L1 FITS file as a Dataset."""
    hdus = fits.open(fn)
    camera = hdus["PRIMARY"].header["CAMERA"].replace("WEAVE", "")
    band = camera[0]
    hdr = hdus[f"{camera}_DATA"].header
    increment, zeropoint, size_wl, size_nspec = (
        hdr["CD1_1"],
        hdr["CRVAL1"],
        hdr["NAXIS1"],
        hdr["NAXIS2"],
    )
    wl = np.arange(0, size_wl) * increment + zeropoint
    nspec = np.arange(1, size_nspec + 1)
    coords = {"NSPEC": nspec, f"LAMBDA_{band}": wl}
    dims = list(coords.keys())
    arrays = {}
    for ext in ["DATA", "IVAR", "DATA_NOSS", "IVAR_NOSS", "SENSFUNC"]:
        name = f"{camera}_{ext}"
        data = hdus[name].data
        unit = hdus[name].header["BUNIT"]
        name = name.replace("DATA", "FLUX")
        arrays[name] = xr.Variable(dims, data, attrs={"unit": str(unit)})
    return xr.Dataset(arrays, coords)


read_l1_data = to_dataset(_l1_data_reader)

In [ ]:
# |hide
show_doc(read_l1_data, name="read_l1_data", title_level=4)

---

[source](https://github.com/bamford/qagmire/blob/main/qagmire/data.py#L356){target="_blank" style="float:right; font-size:smaller"}

#### read_l1_data

>      read_l1_data (fn)

Read the data from a WEAVE L1 FITS file as a Dataset.

In [ ]:
l1_data = read_l1_data(lr_l1_single_files)

Locating and converting where necessary: 100%|██████████| 60/60 [00:21<00:00,  2.84it/s]
Reading netCDF files... took 3.00 s. Size is 27397.908 Mb


In [ ]:
l1_data

<xarray.Dataset>
Dimensions:         (NSPEC: 960, LAMBDA_B: 9649, filename: 60, LAMBDA_R: 15289)
Coordinates:
  * NSPEC           (NSPEC) int64 1 2 3 4 5 6 7 ... 954 955 956 957 958 959 960
  * LAMBDA_B        (LAMBDA_B) float64 3.676e+03 3.676e+03 ... 6.088e+03
  * filename        (filename) <U14 'single_1003320' ... 'single_1003437'
    RUN             (filename) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    CAMERA          (filename) <U4 dask.array<chunksize=(1,), meta=np.ndarray>
    MJD             (filename) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    OBID            (filename) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * LAMBDA_R        (LAMBDA_R) float64 5.772e+03 5.772e+03 ... 9.594e+03
Data variables:
    BLUE_FLUX       (filename, NSPEC, LAMBDA_B) float32 dask.array<chunksize=(1, 960, 9649), meta=np.ndarray>
    BLUE_IVAR       (filename, NSPEC, LAMBDA_B) float32 dask.array<chunksize=(1, 960, 9649), meta=np.ndarray>
    BLUE_FLUX_NOSS  (filename, NSPEC, LAMBDA_B) float32 dask.array<chunksize=(1, 960, 9649), meta=np.ndarray>
    BLUE_IVAR_NOSS  (filename, NSPEC, LAMBDA_B) float32 dask.array<chunksize=(1, 960, 9649), meta=np.ndarray>
    BLUE_SENSFUNC   (filename, NSPEC, LAMBDA_B) float32 dask.array<chunksize=(1, 960, 9649), meta=np.ndarray>
    RED_FLUX        (filename, NSPEC, LAMBDA_R) float32 dask.array<chunksize=(4, 960, 15289), meta=np.ndarray>
    RED_IVAR        (filename, NSPEC, LAMBDA_R) float32 dask.array<chunksize=(4, 960, 15289), meta=np.ndarray>
    RED_FLUX_NOSS   (filename, NSPEC, LAMBDA_R) float32 dask.array<chunksize=(4, 960, 15289), meta=np.ndarray>
    RED_IVAR_NOSS   (filename, NSPEC, LAMBDA_R) float32 dask.array<chunksize=(4, 960, 15289), meta=np.ndarray>
    RED_SENSFUNC    (filename, NSPEC, LAMBDA_R) float32 dask.array<chunksize=(4, 960, 15289), meta=np.ndarray>

It is a good idea to close a `Dataset` when you are finished with it, as otherwise other processes may not be able to access the same underlying files.

In [ ]:
l1_data.close()

### FIBTABLE

The FIBTABLE extension contains information about the fibre allocations, plus some basic measurements.

In [ ]:
# Table.read(l1_hdus["FIBTABLE"])

In [ ]:
l1_fibre_table = read_fibre_table(lr_l1_single_files)

Reading files: 100%|██████████| 60/60 [00:01<00:00, 53.50it/s]
Creating Dataset... took 0.59 s. Size is 18.848 Mb


In [ ]:
l1_fibre_table

<xarray.Dataset>
Dimensions:       (APS_ID: 1004, filename: 60)
Coordinates:
  * APS_ID        (APS_ID) int16 1 2 3 4 5 6 7 ... 1002 1003 1004 1005 1006 1007
  * filename      (filename) object 'single_1003318' ... 'single_1004150'
    RUN           (filename) int64 1003318 1003317 1003329 ... 1004149 1004150
    CAMERA        (filename) <U4 'BLUE' 'RED' 'RED' ... 'BLUE' 'RED' 'BLUE'
    MJD           (filename) float64 5.781e+04 5.781e+04 ... 5.803e+04 5.803e+04
    OBID          (filename) int64 3802 3802 3900 3802 ... 3936 3936 3936 3936
Data variables: (12/59)
    NSPEC         (filename, APS_ID) float32 nan nan nan nan ... nan nan nan nan
    CNAME         (filename, APS_ID) object nan nan nan nan ... nan nan nan nan
    FIBRERA       (filename, APS_ID) float64 nan nan nan nan ... nan nan nan nan
    FIBREDEC      (filename, APS_ID) float64 nan nan nan nan ... nan nan nan nan
    XPOSITION     (filename, APS_ID) float32 nan nan nan nan ... nan nan nan nan
    YPOSITION     (filename, APS_ID) float32 nan nan nan nan ... nan nan nan nan
    ...            ...
    MEANFLUX_G    (filename, APS_ID) float32 nan nan nan nan ... nan nan nan nan
    MEANFLUX_R    (filename, APS_ID) float32 nan nan nan nan ... nan nan nan nan
    MEANFLUX_I    (filename, APS_ID) float32 nan nan nan nan ... nan nan nan nan
    MEANFLUX_GG   (filename, APS_ID) float32 nan nan nan nan ... nan nan nan nan
    MEANFLUX_BP   (filename, APS_ID) float32 nan nan nan nan ... nan nan nan nan
    MEANFLUX_RP   (filename, APS_ID) float32 nan nan nan nan ... nan nan nan nan

## L2 files

These contain higher-level processed data products. There are `single` files, which contain meaurements on a single exposure, and `stack` files, which contain the same measurements on stacked exposures.

We will read some simulated stack files to show examples.

In [ ]:
lr_l2_stack_files = get_lr_l2_stack_files(date="2017*")
print(len(lr_l2_stack_files), "low-res L2 stack files")

6 low-res L2 stack files


In [ ]:
l2_hdus = fits.open(lr_l2_stack_files[0])

WEAVE L2 stack files contain six extensions:

In [ ]:
print([hdu.name for hdu in l2_hdus])

['PRIMARY', 'CLASS_TABLE', 'STAR_TABLE', 'GALAXY_TABLE', 'CLASS_SPEC', 'STAR_SPEC', 'GALAXY_SPEC']


### PRIMARY

The PRIMARY extension contains only a header with some basic information.

In [ ]:
l2_hdus["PRIMARY"].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
L1_REF_0= 'stack_1003318.fit'  / L1 reference file                              
L1_REF_1= 'stack_1003317.fit'  / L1 reference file                              
L1_REF_2= '' / L1 reference file                                                
DATE-OBS= '20170224'           / L1: OBS-DATE                                   
OBSMODE = 'MOS     '           / L1: OBSMODE                                    
RES-OBS = 'LR      '           / L2: RES-DATE                                   
OBID    = '3802    '           / L1: OBID                                       
CHECKSUM= '97WQA7WO97WOA7WO'   / HDU checksum updated 2022-02-08T02:11:32       
DATASUM = '0       '        

In [ ]:
primary_header = read_primary_header(lr_l2_stack_files)

Reading files: 100%|██████████| 6/6 [00:00<00:00, 107.51it/s]
Creating Dataset... took 0.01 s. Size is 0.002 Mb


### CLASS_TABLE

The CLASS_TABLE extension contains information from matching various templates to the spectra. The redshift and class of the best fitting template are given, as well as the full cross-correlation results of each template as a function of redshift.

In [ ]:
# Table.read(l2_hdus["CLASS_TABLE"]) # contains multidimensional columns, so not shown

In [ ]:
# |export


def _class_table_reader(fn):
    """Read the CLASS_TABLE from a WEAVE L2 FITS file as a Dataset.

    All quantities are indexed by the `APS_ID` of the fibre.
    Chi-square values `CZZ_CHI2_*` for each template are further indexed by redshift `CZZ_*`.
    Coefficients `COEFF` and indexed by integers `I_COEFF`.
    """
    cols = _read_fits_columns(fn, "CLASS_TABLE")
    if not cols:
        return None
    coords = dict(APS_ID=cols.pop("APS_ID"))
    # convert CZZ columns to coordinates
    for c in list(cols):
        if c.startswith("CZZ") and "CHI2" not in c:
            czz_all = cols.pop(c)
            czz = czz_all[0]
            assert (czz == czz_all).all()
            coords[c] = czz
    for c in cols:
        dims = ["APS_ID"]
        if c.startswith("CZZ"):
            dims += [c.replace("_CHI2", "")]
        elif c == "COEFF":
            dims += ["I_COEFF"]
        cols[c] = xr.Variable(dims, cols[c], attrs={"unit": str(cols[c].unit)})
    return xr.Dataset(cols, coords)


read_class_table = to_dataset(_class_table_reader)

In [ ]:
# |hide
show_doc(read_class_table, name="read_class_table", title_level=4)

---

[source](https://github.com/bamford/qagmire/blob/main/qagmire/data.py#L385){target="_blank" style="float:right; font-size:smaller"}

#### read_class_table

>      read_class_table (fn)

Read the CLASS_TABLE from a WEAVE L2 FITS file as a Dataset.

All quantities are indexed by the `APS_ID` of the fibre.
Chi-square values `CZZ_CHI2_*` for each template are further indexed by redshift `CZZ_*`.
Coefficients `COEFF` and indexed by integers `I_COEFF`.

In [ ]:
class_table = read_class_table(lr_l2_stack_files)

Locating and converting where necessary: 100%|██████████| 6/6 [00:04<00:00,  1.23it/s]
Reading netCDF files... took 1.15 s. Size is 178.858 Mb


In [ ]:
class_table

<xarray.Dataset>
Dimensions:           (APS_ID: 995, CZZ_GALAXY: 1446, CZZ_QSO: 1648,
                       CZZ_STAR_A: 101, CZZ_STAR_B: 101, CZZ_STAR_CV: 101,
                       CZZ_STAR_F: 101, CZZ_STAR_G: 101, CZZ_STAR_K: 101,
                       CZZ_STAR_M: 101, CZZ_STAR_WD: 101, filename: 6,
                       I_COEFF: 10)
Coordinates: (12/13)
  * APS_ID            (APS_ID) int64 1 2 3 4 5 6 ... 1003 1004 1005 1006 1007
  * CZZ_GALAXY        (CZZ_GALAXY) float64 -0.005 -0.004312 ... 1.698 1.7
  * CZZ_QSO           (CZZ_QSO) float64 0.05 0.05121 0.05242 ... 5.985 5.993
  * CZZ_STAR_A        (CZZ_STAR_A) float64 -0.002 -0.00196 ... 0.00196 0.002
  * CZZ_STAR_B        (CZZ_STAR_B) float64 -0.002 -0.00196 ... 0.00196 0.002
  * CZZ_STAR_CV       (CZZ_STAR_CV) float64 -0.002 -0.00196 ... 0.00196 0.002
    ...                ...
  * CZZ_STAR_G        (CZZ_STAR_G) float64 -0.002 -0.00196 ... 0.00196 0.002
  * CZZ_STAR_K        (CZZ_STAR_K) float64 -0.002 -0.00196 ... 0.00196 0.002
  * CZZ_STAR_M        (CZZ_STAR_M) float64 -0.002 -0.00196 ... 0.00196 0.002
  * CZZ_STAR_WD       (CZZ_STAR_WD) float64 -0.002 -0.00196 ... 0.00196 0.002
  * filename          (filename) <U32 'stack_1003330__stack_1003329_APS' ... ...
    OBID              (filename) <U4 dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: I_COEFF
Data variables: (12/25)
    TARGID            (filename, APS_ID) object dask.array<chunksize=(1, 995), meta=np.ndarray>
    CNAME             (filename, APS_ID) object dask.array<chunksize=(1, 995), meta=np.ndarray>
    Z                 (filename, APS_ID) float64 dask.array<chunksize=(1, 995), meta=np.ndarray>
    ZERR              (filename, APS_ID) float64 dask.array<chunksize=(1, 995), meta=np.ndarray>
    ZWARN             (filename, APS_ID) float64 dask.array<chunksize=(1, 995), meta=np.ndarray>
    CLASS             (filename, APS_ID) object dask.array<chunksize=(1, 995), meta=np.ndarray>
    ...                ...
    CZZ_CHI2_STAR_CV  (filename, APS_ID, CZZ_STAR_CV) float64 dask.array<chunksize=(1, 995, 101), meta=np.ndarray>
    CZZ_CHI2_STAR_F   (filename, APS_ID, CZZ_STAR_F) float64 dask.array<chunksize=(1, 995, 101), meta=np.ndarray>
    CZZ_CHI2_STAR_G   (filename, APS_ID, CZZ_STAR_G) float64 dask.array<chunksize=(1, 995, 101), meta=np.ndarray>
    CZZ_CHI2_STAR_K   (filename, APS_ID, CZZ_STAR_K) float64 dask.array<chunksize=(1, 995, 101), meta=np.ndarray>
    CZZ_CHI2_STAR_M   (filename, APS_ID, CZZ_STAR_M) float64 dask.array<chunksize=(1, 995, 101), meta=np.ndarray>
    CZZ_CHI2_STAR_WD  (filename, APS_ID, CZZ_STAR_WD) float64 dask.array<chunksize=(1, 995, 101), meta=np.ndarray>

In [ ]:
class_table.close()

### STAR_TABLE

The STAR_TABLE extension contains measurements of stellar parameters.

In [ ]:
# Table.read(l2_hdus["STAR_TABLE"])  # contains multidimensional columns, so not shown

In [ ]:
# |export


def _star_table_reader(fn):
    """Read the STAR_TABLE from a WEAVE L2 FITS file as a Dataset.

    All quantities are indexed by the `APS_ID` of the fibre.
    The covariance matrix `COVAR` is additionally indexed by `I_COVAR`, `J_COVAR`.
    The elements `ELEM` and `ELEM_ERR` are additionally indexed by `I_ELEM`.
    """
    cols = _read_fits_columns(fn, "STAR_TABLE")
    if not cols:
        return None
    coords = dict(APS_ID=cols.pop("APS_ID"))
    coords["I_COVAR"] = coords["J_COVAR"] = ["TEFF", "LOGG", "FEH", "ALPHA", "MICRO"]
    for c in cols:
        dims = ["APS_ID"]
        if c == "COVAR":
            dims += ["I_COVAR", "J_COVAR"]
        elif "ELEM" in c:
            dims += ["I_ELEM"]
        cols[c] = xr.Variable(dims, cols[c], attrs={"unit": str(cols[c].unit)})
    return xr.Dataset(cols, coords)


read_star_table = to_dataset(_star_table_reader)

In [ ]:
# |hide
show_doc(read_star_table, name="read_star_table", title_level=4)

---

[source](https://github.com/bamford/qagmire/blob/main/qagmire/data.py#L416){target="_blank" style="float:right; font-size:smaller"}

#### read_star_table

>      read_star_table (fn)

Read the STAR_TABLE from a WEAVE L2 FITS file as a Dataset.

All quantities are indexed by the `APS_ID` of the fibre.
The covariance matrix `COVAR` is additionally indexed by `I_COVAR`, `J_COVAR`.
The elements `ELEM` and `ELEM_ERR` are additionally indexed by `I_ELEM`.

In [ ]:
star_table = read_star_table(lr_l2_stack_files)

Locating and converting where necessary: 100%|██████████| 6/6 [00:00<00:00, 21.32it/s]
Reading netCDF files... took 1.05 s. Size is 0.065 Mb


In [ ]:
star_table

<xarray.Dataset>
Dimensions:        (APS_ID: 24, I_COVAR: 5, J_COVAR: 5, filename: 6, I_ELEM: 2)
Coordinates:
  * APS_ID         (APS_ID) int64 194 222 283 304 329 ... 897 922 946 948 992
  * I_COVAR        (I_COVAR) <U5 'TEFF' 'LOGG' 'FEH' 'ALPHA' 'MICRO'
  * J_COVAR        (J_COVAR) <U5 'TEFF' 'LOGG' 'FEH' 'ALPHA' 'MICRO'
  * filename       (filename) <U32 'stack_1003354__stack_1003353_APS' ... 'st...
    OBID           (filename) <U4 dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: I_ELEM
Data variables: (12/33)
    TARGID         (filename, APS_ID) object dask.array<chunksize=(1, 24), meta=np.ndarray>
    CNAME          (filename, APS_ID) object dask.array<chunksize=(1, 24), meta=np.ndarray>
    VRAD           (filename, APS_ID) float64 dask.array<chunksize=(1, 24), meta=np.ndarray>
    VRAD_ERR       (filename, APS_ID) float64 dask.array<chunksize=(1, 24), meta=np.ndarray>
    SKEWNESS_RVS   (filename, APS_ID) float64 dask.array<chunksize=(1, 24), meta=np.ndarray>
    KURTOSIS_RVS   (filename, APS_ID) float64 dask.array<chunksize=(1, 24), meta=np.ndarray>
    ...             ...
    COVAR          (filename, APS_ID, I_COVAR, J_COVAR) float64 dask.array<chunksize=(1, 24, 5, 5), meta=np.ndarray>
    ELEM           (filename, APS_ID, I_ELEM) float64 dask.array<chunksize=(1, 24, 2), meta=np.ndarray>
    ELEM_ERR       (filename, APS_ID, I_ELEM) float64 dask.array<chunksize=(1, 24, 2), meta=np.ndarray>
    SNR_FR         (filename, APS_ID) float64 dask.array<chunksize=(1, 24), meta=np.ndarray>
    CHISQ_TOT      (filename, APS_ID) float64 dask.array<chunksize=(1, 24), meta=np.ndarray>
    FLAG_FR        (filename, APS_ID) float64 dask.array<chunksize=(1, 24), meta=np.ndarray>

In [ ]:
star_table.close()

### GALAXY_TABLE

The GALAXY_TABLE extension contains measurements of galaxy parameters, including Hubble-flow corrected redshifts, stellar kinematics, line fits and indices.

In [ ]:
# Table.read(l2_hdus["GALAXY_TABLE"], unit_parse_strict="silent")  # contains multidimensional columns, so not shown

In [ ]:
# |export


def _not_line_col(c):
    """Identify columns that do not contain line measurements."""
    c = c.replace("ERR_", "")
    for n in ["EBMV0", "EBMV1", "FLUX", "AMPL", "Z", "SIGMA", "AON", "FWHM"]:
        if c.startswith(n + "_"):
            return False
    return True


def _process_line_quantities(cols, lines):
    """Process line quantities.

    Quantities with multiple elements are split into separate columns.

    The supplied `cols` dictionary is modified in-place.
    """
    line_quantities = []
    for c in list(cols):
        if c.endswith(lines[0]):
            qty = c.replace("_" + lines[0], "")
            ndim = cols[c].ndim
            if ndim == 1:
                line_quantities.append(qty)
            elif ndim == 2:
                nq = cols[c].shape[1]
                for i in range(nq):
                    line_quantities.append(f"{qty}{i}")
                for line in lines:
                    oldcol = cols.pop(f"{qty}_{line}")
                    for i in range(nq):
                        cols[f"{qty}{i}_{line}"] = oldcol[:, i]
    return line_quantities


def _galaxy_table_reader(fn):
    """Read the GALAXY_TABLE from a WEAVE L2 FITS file as a Dataset.

    All quantities are indexed by the `APS_ID` of the fibre.
    The line measurements are additionally indexed by the measurement quantity `QTY`
    and the line name `LINE`.
    The index measurements are additionally indexed by the index name `INDEX`.
    """
    # TODO: add units where missing
    cols = _read_fits_columns(fn, "GALAXY_TABLE")
    if not cols:
        return None
    coords = dict(APS_ID=cols.pop("APS_ID"))
    coords["LINE"] = [c.replace("FLUX_", "") for c in cols if c.startswith("FLUX")]
    coords["QTY"] = _process_line_quantities(cols, coords["LINE"])
    coords["INDEX"] = [c for i, c in enumerate(cols) if (i > 100 and _not_line_col(c))]
    line_cols = [
        [cols.pop(f"{qty}_{line}") for line in coords["LINE"]] for qty in coords["QTY"]
    ]
    index_cols = [cols.pop(idx) for idx in coords["INDEX"]]
    out_cols = {}
    for i, c in enumerate(cols):
        dims = ["APS_ID"]
        out_cols[c] = xr.Variable(dims, cols[c], attrs={"unit": str(cols[c].unit)})
    out_cols["LINES"] = xr.Variable(["QTY", "LINE", "APS_ID"], line_cols)
    out_cols["INDICES"] = xr.Variable(["INDEX", "APS_ID"], index_cols)
    return xr.Dataset(out_cols, coords)


read_galaxy_table = to_dataset(_galaxy_table_reader)

In [ ]:
# |hide
show_doc(read_galaxy_table, name="read_galaxy_table", title_level=4)

---

[source](https://github.com/bamford/qagmire/blob/main/qagmire/data.py#L475){target="_blank" style="float:right; font-size:smaller"}

#### read_galaxy_table

>      read_galaxy_table (fn)

Read the GALAXY_TABLE from a WEAVE L2 FITS file as a Dataset.

All quantities are indexed by the `APS_ID` of the fibre.
The line measurements are additionally indexed by the measurement quantity `QTY`
and the line name `LINE`.
The index measurements are additionally indexed by the index name `INDEX`.

In [ ]:
galaxy_table = read_galaxy_table(lr_l2_stack_files)

Locating and converting where necessary: 100%|██████████| 6/6 [00:01<00:00,  4.51it/s]
Reading netCDF files... took 1.14 s. Size is 27.393 Mb


In [ ]:
galaxy_table

<xarray.Dataset>
Dimensions:         (APS_ID: 994, LINE: 22, QTY: 13, INDEX: 292, filename: 6)
Coordinates:
  * APS_ID          (APS_ID) int32 1 2 3 4 5 6 ... 1002 1003 1004 1005 1006 1007
  * LINE            (LINE) <U15 'HeII_3203.15' ... '[ArIII]_7135.67'
  * QTY             (QTY) <U9 'FLUX' 'AMPL' 'Z' ... 'ERR_EBMV0' 'ERR_EBMV1'
  * INDEX           (INDEX) <U16 'BL1719' 'ERR_BL1719' ... 'ERR_MgI2.28'
  * filename        (filename) <U32 'stack_1003354__stack_1003353_APS' ... 's...
    OBID            (filename) <U4 dask.array<chunksize=(1,), meta=np.ndarray>
Data variables: (12/26)
    TARGID          (filename, APS_ID) object dask.array<chunksize=(1, 994), meta=np.ndarray>
    CNAME           (filename, APS_ID) object dask.array<chunksize=(1, 994), meta=np.ndarray>
    ZCORR           (filename, APS_ID) float64 dask.array<chunksize=(1, 994), meta=np.ndarray>
    V               (filename, APS_ID) float64 dask.array<chunksize=(1, 994), meta=np.ndarray>
    SIGMA           (filename, APS_ID) float64 dask.array<chunksize=(1, 994), meta=np.ndarray>
    H3              (filename, APS_ID) float64 dask.array<chunksize=(1, 994), meta=np.ndarray>
    ...              ...
    FORM_ERR_H4     (filename, APS_ID) float64 dask.array<chunksize=(1, 994), meta=np.ndarray>
    FORM_ERR_H5     (filename, APS_ID) float64 dask.array<chunksize=(1, 994), meta=np.ndarray>
    FORM_ERR_H6     (filename, APS_ID) float64 dask.array<chunksize=(1, 994), meta=np.ndarray>
    FWHM_FLAG       (filename, APS_ID) float32 dask.array<chunksize=(1, 994), meta=np.ndarray>
    LINES           (filename, QTY, LINE, APS_ID) float64 dask.array<chunksize=(1, 13, 22, 994), meta=np.ndarray>
    INDICES         (filename, INDEX, APS_ID) float64 dask.array<chunksize=(1, 292, 994), meta=np.ndarray>

In [ ]:
galaxy_table.close()

### CLASS_SPEC

The CLASS_SPEC extension contains spectra and model fits, performed separately for the red and blue arms.

In [ ]:
# Table.read(l2_hdus["CLASS_SPEC"], unit_parse_strict="silent")  # contains multidimensional columns, so not shown

In [ ]:
# |export


def _class_spec_reader(fn):
    """Read the CLASS_SPEC from a WEAVE L2 FITS file as a Dataset.

    All quantities are indexed by the `APS_ID` of the fibre.
    Spectral quantities are additionally indexed by wavelength `LAMBDA_{B,R}`.
    """
    cols = _read_fits_columns(fn, "CLASS_SPEC", limit_precision=True)
    if not cols:
        return None
    coords = dict(APS_ID=cols.pop("APS_ID"))
    for c in list(cols):
        if c.startswith("LAMBDA"):
            band = c.split("_")[-1]
            wl_all = cols.pop(c)
            wl = wl_all[0]
            assert (wl == wl_all).all()
            coords[f"LAMBDA_{band}"] = wl
    for c in cols:
        dims = ["APS_ID"]
        if c.endswith("_B"):
            dims += ["LAMBDA_B"]
        elif c.endswith("_R"):
            dims += ["LAMBDA_R"]
        cols[c] = xr.Variable(dims, cols[c], attrs={"unit": str(cols[c].unit)})
    return xr.Dataset(cols, coords)


read_class_spec = to_dataset(_class_spec_reader)

In [ ]:
# |hide
show_doc(read_class_spec, name="read_class_spec", title_level=4)

---

[source](https://github.com/bamford/qagmire/blob/main/qagmire/data.py#L507){target="_blank" style="float:right; font-size:smaller"}

#### read_class_spec

>      read_class_spec (fn)

Read the CLASS_SPEC from a WEAVE L2 FITS file as a Dataset.

All quantities are indexed by the `APS_ID` of the fibre.
Spectral quantities are additionally indexed by wavelength `LAMBDA_{B,R}`.

In [ ]:
class_spec = read_class_spec(lr_l2_stack_files)

Locating and converting where necessary: 100%|██████████| 6/6 [00:08<00:00,  1.42s/it]
Reading netCDF files... took 0.49 s. Size is 1703.853 Mb


In [ ]:
class_spec

<xarray.Dataset>
Dimensions:     (APS_ID: 995, LAMBDA_B: 9648, LAMBDA_R: 15288, filename: 6)
Coordinates:
  * APS_ID      (APS_ID) int64 1 2 3 4 5 6 7 ... 1002 1003 1004 1005 1006 1007
  * LAMBDA_B    (LAMBDA_B) float32 3.677e+03 3.677e+03 ... 6.089e+03 6.089e+03
  * LAMBDA_R    (LAMBDA_R) float32 5.774e+03 5.774e+03 ... 9.596e+03 9.596e+03
  * filename    (filename) <U32 'stack_1003330__stack_1003329_APS' ... 'stack...
    OBID        (filename) <U4 dask.array<chunksize=(1,), meta=np.ndarray>
Data variables:
    TARGID      (filename, APS_ID) object dask.array<chunksize=(1, 995), meta=np.ndarray>
    CNAME       (filename, APS_ID) object dask.array<chunksize=(1, 995), meta=np.ndarray>
    FLUX_RR_B   (filename, APS_ID, LAMBDA_B) float32 dask.array<chunksize=(1, 995, 9648), meta=np.ndarray>
    IVAR_RR_B   (filename, APS_ID, LAMBDA_B) float32 dask.array<chunksize=(1, 995, 9648), meta=np.ndarray>
    MODEL_RR_B  (filename, APS_ID, LAMBDA_B) float32 dask.array<chunksize=(1, 995, 9648), meta=np.ndarray>
    FLUX_RR_R   (filename, APS_ID, LAMBDA_R) float32 dask.array<chunksize=(1, 995, 15288), meta=np.ndarray>
    IVAR_RR_R   (filename, APS_ID, LAMBDA_R) float32 dask.array<chunksize=(1, 995, 15288), meta=np.ndarray>
    MODEL_RR_R  (filename, APS_ID, LAMBDA_R) float32 dask.array<chunksize=(1, 995, 15288), meta=np.ndarray>

In [ ]:
class_spec.close()

### STAR_SPEC

The STAR_SPEC extension contains spectra and model fits for stellar measurements.

In [ ]:
# Table.read(l2_hdus["STAR_SPEC"], unit_parse_strict="silent")

In [ ]:
# |export


def _star_spec_reader(fn):
    """Read the STAR_SPEC from a WEAVE L2 FITS file as a Dataset.

    All quantities are indexed by the `APS_ID` of the fibre.
    Spectral quantities are additionally indexed by wavelength bin `LAMBIN_{R,B,C}`,
    which does *not* correspond to the same wavelength for each spectrum.
    """
    cols = _read_fits_columns(fn, "STAR_SPEC", limit_precision=True)
    if not cols:
        return None
    coords = dict(APS_ID=cols.pop("APS_ID"))
    for c in cols:
        dims = ["APS_ID"]
        if c.endswith("_B"):
            dims += ["LAMBIN_B"]
        elif c.endswith("_R"):
            dims += ["LAMBIN_R"]
        elif c.endswith("_C"):
            dims += ["LAMBIN_C"]
        cols[c] = xr.Variable(dims, cols[c], attrs={"unit": str(cols[c].unit)})
    return xr.Dataset(cols, coords)


read_star_spec = to_dataset(_star_spec_reader)

In [ ]:
# |hide
show_doc(read_star_spec, name="read_star_spec", title_level=4)

---

[source](https://github.com/bamford/qagmire/blob/main/qagmire/data.py#L537){target="_blank" style="float:right; font-size:smaller"}

#### read_star_spec

>      read_star_spec (fn)

Read the STAR_SPEC from a WEAVE L2 FITS file as a Dataset.

All quantities are indexed by the `APS_ID` of the fibre.
Spectral quantities are additionally indexed by wavelength bin `LAMBIN_{R,B,C}`,
which does *not* correspond to the same wavelength for each spectrum.

In [ ]:
star_spec = read_star_spec(lr_l2_stack_files)

Locating and converting where necessary: 100%|██████████| 6/6 [00:01<00:00,  4.43it/s]
Reading netCDF files... took 0.83 s. Size is 106.808 Mb


In [ ]:
star_spec

<xarray.Dataset>
Dimensions:       (APS_ID: 24, filename: 6, LAMBIN_B: 9648, LAMBIN_R: 15288,
                   LAMBIN_C: 23672)
Coordinates:
  * APS_ID        (APS_ID) int64 194 222 283 304 329 346 ... 897 922 946 948 992
  * filename      (filename) <U32 'stack_1003342__stack_1003341_APS' ... 'sta...
    OBID          (filename) <U4 dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: LAMBIN_B, LAMBIN_R, LAMBIN_C
Data variables: (12/14)
    TARGID        (filename, APS_ID) object dask.array<chunksize=(1, 24), meta=np.ndarray>
    CNAME         (filename, APS_ID) object dask.array<chunksize=(1, 24), meta=np.ndarray>
    LAMBDA_RVS_B  (filename, APS_ID, LAMBIN_B) float32 dask.array<chunksize=(1, 24, 9648), meta=np.ndarray>
    FLUX_RVS_B    (filename, APS_ID, LAMBIN_B) float32 dask.array<chunksize=(1, 24, 9648), meta=np.ndarray>
    ERROR_RVS_B   (filename, APS_ID, LAMBIN_B) float32 dask.array<chunksize=(1, 24, 9648), meta=np.ndarray>
    MODEL_RVS_B   (filename, APS_ID, LAMBIN_B) float32 dask.array<chunksize=(1, 24, 9648), meta=np.ndarray>
    ...            ...
    ERROR_RVS_R   (filename, APS_ID, LAMBIN_R) float32 dask.array<chunksize=(1, 24, 15288), meta=np.ndarray>
    MODEL_RVS_R   (filename, APS_ID, LAMBIN_R) float32 dask.array<chunksize=(1, 24, 15288), meta=np.ndarray>
    LAMBDA_FR_C   (filename, APS_ID, LAMBIN_C) float32 dask.array<chunksize=(1, 24, 23672), meta=np.ndarray>
    FLUX_FR_C     (filename, APS_ID, LAMBIN_C) float32 dask.array<chunksize=(1, 24, 23672), meta=np.ndarray>
    ERROR_FR_C    (filename, APS_ID, LAMBIN_C) float32 dask.array<chunksize=(1, 24, 23672), meta=np.ndarray>
    MODEL_FR_C    (filename, APS_ID, LAMBIN_C) float32 dask.array<chunksize=(1, 24, 23672), meta=np.ndarray>

In [ ]:
star_spec.close()

### GALAXY_SPEC

The GALAXY_SPEC extension contains log-wavelength-binned spectra and model fits by PPXF and GANDALF.

In [ ]:
# Table.read(l2_hdus["GALAXY_SPEC"], unit_parse_strict="silent")

In [ ]:
# |export


def _galaxy_spec_reader(fn):
    """Read the GALAXY_SPEC from a WEAVE L2 FITS file as a Dataset.

    All quantities are indexed by the `APS_ID` of the fibre.
    Spectral quantities are additionally indexed by log-wavelength bin `LOGLAMBIN`,
    which does *not* correspond to the same wavelength for each spectrum.
    """
    cols = _read_fits_columns(fn, "GALAXY_SPEC", limit_precision=True)
    if not cols:
        return None
    coords = dict(APS_ID=cols.pop("APS_ID"))
    for c in cols:
        dims = ["APS_ID"]
        if c.endswith("_PPXF") or c.endswith("_GAND"):
            dims += ["LOGLAMBIN"]
        cols[c] = xr.Variable(dims, cols[c], attrs={"unit": str(cols[c].unit)})
    return xr.Dataset(cols, coords)


read_galaxy_spec = to_dataset(_galaxy_spec_reader)

In [ ]:
# |hide
show_doc(read_galaxy_spec, name="read_galaxy_spec", title_level=4)

---

[source](https://github.com/bamford/qagmire/blob/main/qagmire/data.py#L563){target="_blank" style="float:right; font-size:smaller"}

#### read_galaxy_spec

>      read_galaxy_spec (fn)

Read the GALAXY_SPEC from a WEAVE L2 FITS file as a Dataset.

All quantities are indexed by the `APS_ID` of the fibre.
Spectral quantities are additionally indexed by log-wavelength bin `LOGLAMBIN`,
which does *not* correspond to the same wavelength for each spectrum.

In [ ]:
galaxy_spec = read_galaxy_spec(lr_l2_stack_files)

Locating and converting where necessary: 100%|██████████| 6/6 [00:37<00:00,  6.30s/it]
Reading netCDF files... took 0.74 s. Size is 10232.270 Mb


In [ ]:
galaxy_spec

<xarray.Dataset>
Dimensions:           (APS_ID: 994, filename: 6, LOGLAMBIN: 23671)
Coordinates:
  * APS_ID            (APS_ID) int32 1 2 3 4 5 6 ... 1003 1004 1005 1006 1007
  * filename          (filename) <U32 'stack_1003330__stack_1003329_APS' ... ...
    OBID              (filename) <U4 dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: LOGLAMBIN
Data variables: (12/15)
    TARGID            (filename, APS_ID) object dask.array<chunksize=(1, 994), meta=np.ndarray>
    CNAME             (filename, APS_ID) object dask.array<chunksize=(1, 994), meta=np.ndarray>
    LOGLAM_PPXF       (filename, APS_ID, LOGLAMBIN) float32 dask.array<chunksize=(1, 994, 23671), meta=np.ndarray>
    FLUX_PPXF         (filename, APS_ID, LOGLAMBIN) float32 dask.array<chunksize=(1, 994, 23671), meta=np.ndarray>
    ERROR_PPXF        (filename, APS_ID, LOGLAMBIN) float32 dask.array<chunksize=(1, 994, 23671), meta=np.ndarray>
    MODEL_PPXF        (filename, APS_ID, LOGLAMBIN) float32 dask.array<chunksize=(1, 994, 23671), meta=np.ndarray>
    ...                ...
    ERROR_GAND        (filename, APS_ID, LOGLAMBIN) float32 dask.array<chunksize=(1, 994, 23671), meta=np.ndarray>
    MODEL_GAND        (filename, APS_ID, LOGLAMBIN) float64 dask.array<chunksize=(1, 994, 23671), meta=np.ndarray>
    EMISSION_GAND     (filename, APS_ID, LOGLAMBIN) float64 dask.array<chunksize=(1, 994, 23671), meta=np.ndarray>
    FLUX_CLEAN_GAND   (filename, APS_ID, LOGLAMBIN) float64 dask.array<chunksize=(1, 994, 23671), meta=np.ndarray>
    MODEL_CLEAN_GAND  (filename, APS_ID, LOGLAMBIN) float64 dask.array<chunksize=(1, 994, 23671), meta=np.ndarray>
    GOODPIX_GAND      (filename, APS_ID, LOGLAMBIN) float64 dask.array<chunksize=(1, 994, 23671), meta=np.ndarray>

In [ ]:
galaxy_spec.close()

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()